# Homework: 使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少

In [1]:
# 数据集加载和处理
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")


Reusing dataset yelp_review_full (/root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    
    


In [3]:
# 数据预处理
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

show_random_elements(tokenized_datasets["train"], num_examples=1)

# 使用全量数据计算，仅进行乱序

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

Loading cached processed dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-bdeeeb5eac0c8a8b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-29089685913b5149.arrow


,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,Finally a burger place to replace that greasy burger joint we are all used to! Great tasting homemade burgers. The fries are so fresh. Kids love this place because no one cares if they are talking to loud. Everyone seems happy to be there and you can order online to avoid a line! And yes there are free peanuts!,"[101, 4428, 170, 171, 23872, 1282, 1106, 4971, 1115, 176, 11811, 5821, 171, 23872, 4091, 1195, 1132, 1155, 1215, 1106, 106, 2038, 21296, 1313, 12540, 171, 23872, 1116, 119, 1109, 175, 3377, 1132, 1177, 4489, 119, 9904, 1567, 1142, 1282, 1272, 1185, 1141, 16903, 1191, 1152, 1132, 2520, 1106, 4632, 119, 6064, 3093, 2816, 1106, 1129, 1175, 1105, 1128, 1169, 1546, 3294, 1106, 3644, 170, 1413, 106, 1262, 4208, 1175, 1132, 1714, 185, 23629, 1116, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-20dce6b5ce11b893.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-3ef04ad90c3425ab.arrow


In [4]:
# 加载模型
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [5]:
# 模型训练器
import os
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import wandb
os.environ["WANDB_PROJECT"]="fine-tune-quickstart"
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="false"


model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  per_device_train_batch_size=96,
                                  per_device_eval_batch_size=96,
                                  bf16=True,
                                  save_total_limit=2,
                                  hub_strategy="checkpoint",
                                  save_strategy="steps",
                                  save_steps=500,
                                  ignore_data_skip=True,
                                  report_to="wandb",
                                  logging_steps=5,
                                 )

# 定义模型评估函数
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)




trainer = Trainer(
    model=model,
    
     
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


In [6]:
trainer.train(resume_from_checkpoint=True)
wandb.finish()

Loading model from models/bert-base-cased/test_trainer/checkpoint-5500.
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 650000
  Num Epochs = 3
  Instantaneous batch size per device = 96
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 20313
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Con

Epoch,Training Loss,Validation Loss,Accuracy
1,0.694300,0.693678,0.694380
2,0.615100,0.717382,0.696680
2,0.510600,0.753520,0.694020


Saving model checkpoint to models/bert-base-cased/test_trainer/checkpoint-6000
Configuration saved in models/bert-base-cased/test_trainer/checkpoint-6000/config.json
Model weights saved in models/bert-base-cased/test_trainer/checkpoint-6000/pytorch_model.bin
Deleting older checkpoint [models/bert-base-cased/test_trainer/checkpoint-5400] due to args.save_total_limit
Saving model checkpoint to models/bert-base-cased/test_trainer/checkpoint-6500
Configuration saved in models/bert-base-cased/test_trainer/checkpoint-6500/config.json
Model weights saved in models/bert-base-cased/test_trainer/checkpoint-6500/pytorch_model.bin
Deleting older checkpoint [models/bert-base-cased/test_trainer/checkpoint-5500] due to args.save_total_limit
Saving model checkpoint to models/bert-base-cased/test_trainer/checkpoint-7000
Configuration saved in models/bert-base-cased/test_trainer/checkpoint-7000/config.json
Model weights saved in models/bert-base-cased/test_trainer/checkpoint-7000/pytorch_model.bin
Delet

eval/accuracy,▂█▁
eval/loss,▁▄█
eval/runtime,▁██
eval/samples_per_second,█▁▁
eval/steps_per_second,█▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▄▅▆▅▅▅▄▅▅▄▇▅▆▄▆▇▇█▄▅▅▄▄▂▄▃▄▄▃▅▄▂▃▃▃▃▃▁▂▃
train/total_flos,▁
train/train_loss,▁


In [7]:
# 验证


small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(500))
trainer.evaluate(small_test_dataset)


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 96


Error: You must call wandb.init() before wandb.log()

In [8]:
trainer.save_model(f"{model_dir}/finetuned-trainer")
trainer.save_state()

Saving model checkpoint to models/bert-base-cased/finetuned-trainer
Configuration saved in models/bert-base-cased/finetuned-trainer/config.json
Model weights saved in models/bert-base-cased/finetuned-trainer/pytorch_model.bin


In [ ]:
print(len(tokenized_datasets["test"]))